In [ ]:
import os

os.chdir("../")

import pandas as pd
import numpy as np
from src.data.DatasetConstruction import *
from src.methods.utils.isolation_forest  import *

In [ ]:
ntw = load_elliptic()
ntw_torch = ntw.get_network_torch()
X = ntw_torch.x
y = ntw_torch.y

X = X[:,1:94]
X_train = X.numpy(); print('Shape of X_train:', X_train.shape)
y_train = y.numpy()

In [ ]:
clf = IsolationForest(
    bootstrap=True)
clf.fit(X_train)

In [ ]:
scores = True
if scores:
    y_scores = clf.score_samples(X_train)
else:
    y_scores = clf.predict(X_train) 
y_min = y_scores.min()
y_max = y_scores.max()

bins = np.linspace(y_min, y_max, 50)

In [ ]:
df_scores_labels = pd.DataFrame({'label': y_train, 'score':y_scores})
df_scores_labeled = df_scores_labels[df_scores_labels['label']<2]

In [ ]:
import matplotlib.pyplot as plt

# Separate the scores by label
scores_label_0 = df_scores_labels[df_scores_labels['label'] == 0]['score']
scores_label_1 = df_scores_labels[df_scores_labels['label'] == 1]['score']
scores_label_2 = df_scores_labels[df_scores_labels['label'] == 2]['score']

# Normalize the histograms
plt.hist(scores_label_0, bins=bins, alpha=0.4, label='Legit', color='blue', density=True)
plt.hist(scores_label_1, bins=bins, alpha=0.4, label='Money Laundering', color='green', density=True)
#plt.hist(scores_label_2, bins=bins, alpha=0.4, label='Unknown', color='red', density=True)

# Add labels and legend
plt.xlabel('Anomaly Score')
plt.ylabel('Density')
if scores:
    title = 'Normalized Histogram of Anomaly Scores by True Label'
else:
    title = 'Normalized Histogram of Predicted Labels by True Label'
plt.title(title)
plt.legend(loc='upper right')

# Show the plot
#plt.savefig("misc/isolation_forest_histogram_normalized"+str(scores)+".png")
plt.close()

# Plot the histograms
plt.hist(scores_label_0, bins=bins, alpha=0.4, label='Legit', color='blue')
plt.hist(scores_label_1, bins=bins, alpha=0.4, label='Money Laundering', color='green')
#plt.hist(scores_label_2, bins=bins, alpha=0.4, label='Unknown', color='red')

# Add labels and legend
plt.xlabel('Anomaly Score')
plt.ylabel('Count (log scale)')

if scores:
    title = 'Histogram of Anomaly Scores by True Label'
else:
    title = 'Histogram of Predicted Labels by True Label'
plt.title(title)    
plt.legend(loc='upper right')

# Show the plot
plt.savefig("misc/isolation_forest_histogram"+str(scores)+".pdf")